In [14]:
import pandas as pd
import datetime

In [15]:
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet")

# REGLAS CONTAR VP

In [16]:
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"]) #formato fecha
df_vp["mes"]  = df_vp["fecha"].dt.month #agregar mes
df_vp["ano"]  = df_vp["fecha"].dt.year #agregar año
df_vp["dia"]  = df_vp["fecha"].dt.day #agregar año


pd.set_option("display.max_rows", None)   # muestra todas las filas
pd.set_option("display.max_colwidth", None)  # muestra texto completo en celdas

df_perdidas = df_vp[df_vp["cantidad"] > df_vp["cantidad_existencia"]] # ventas peridas realmente
df_perdidas["vp"] = df_vp["cantidad"] - df_vp["cantidad_existencia"]


# REGLAS FLTRADO DE VENTAS PERDIDAS
vp_filtradas = df_perdidas[(df_perdidas["cantidad"]>=1)  & (df_perdidas["cantidad"]<=99)].copy()
vp_filtradas.sort_values(by=["vp"], ascending=False).head(5)

#vp_filtradas = vp_filtradas[~vp_filtradas["cliente_name"].str.contains("PROMOCION", case=False, na=False)].copy() #Solicitar nueva regla retirar ventas perdidas para cliente promocion esta duplicandolas

vp_def = vp_filtradas[["store_name","product_id","origen","cliente_name","fecha","vp","mes","dia","ano"]]

/tmp/ipykernel_118733/1854185918.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perdidas["vp"] = df_vp["cantidad"] - df_vp["cantidad_existencia"]


In [28]:
#vp_filtradas[vp_filtradas["cliente_name"].str.contains("PROMOCION")].head(10)
vp_filtradas[vp_filtradas["fecha"]=="2025-07-10"].head(10)

,product_ref,cantidad,fecha,cantidad_existencia,origen,product_id,product_name,cliente_name,store_id_num,store_name,mes,ano,dia,vp
276050,DAR12276025,1,2025-07-10,0,cotizacion,"[50310, '[DAR12276025] DA2276UHE FILTRO AIRE ALTA EFICIENCIA - PERKINS - TEREX (025 DA2276UHE)']",[DAR12276025] DA2276UHE FILTRO AIRE ALTA EFICIENCIA - PERKINS - TEREX (025 DA2276UHE),[CO8301292898] SOLARTE NACIONAL DE CONSTRUCCIONES SAS,1,PRINCIPAL COTA,7,2025,10,1
276051,DAE02276025,1,2025-07-10,0,cotizacion,"[16918, '[DAE02276025] DA2276 FILTRO AIRE PERKINS, TEREX (025 DA2276)']","[DAE02276025] DA2276 FILTRO AIRE PERKINS, TEREX (025 DA2276)",[CO8301292898] SOLARTE NACIONAL DE CONSTRUCCIONES SAS,1,PRINCIPAL COTA,7,2025,10,1
276055,BHS00322125,1,2025-07-10,0,cotizacion,"[19782, '[BHS00322125] GS322 FILTRO HIDRAULICO CARGADOR J.DEERE AT189924 (125 BT9347MPG)']",[BHS00322125] GS322 FILTRO HIDRAULICO CARGADOR J.DEERE AT189924 (125 BT9347MPG),[CO8301292898] SOLARTE NACIONAL DE CONSTRUCCIONES SAS,1,PRINCIPAL COTA,7,2025,10,1
276067,BHS00496125,1,2025-07-10,0,cotizacion,"[19758, '[BHS00496125] GS496 FILTRO HIDRAULICO CATERPILLAR 1850337 (125 BT8886MPG)']",[BHS00496125] GS496 FILTRO HIDRAULICO CATERPILLAR 1850337 (125 BT8886MPG),[CO191461130] MARIO ALBERTO HUERTAS COTES,1,PRINCIPAL COTA,7,2025,10,1
276070,DCS00331189,2,2025-07-10,0,cotizacion,"[47436, '[DCS00331189] GS331 FILTRO SEPARADOR COMBUSTIBLE - NISSAN (189 GS331)']",[DCS00331189] GS331 FILTRO SEPARADOR COMBUSTIBLE - NISSAN (189 GS331),[CO191461130] MARIO ALBERTO HUERTAS COTES,1,PRINCIPAL COTA,7,2025,10,2
276082,DAR02239025,1,2025-07-10,0,cotizacion,"[16896, '[DAR02239025] DA2239 FILTRO AIRE BRIGADIER MOTOR CUMMINS (025 DA2239)']",[DAR02239025] DA2239 FILTRO AIRE BRIGADIER MOTOR CUMMINS (025 DA2239),[CC222222222221] CONSUMIDOR FINAL CALLE 6,2,SUCURSAL CALLE 6,7,2025,10,1
276090,DAR02523025,1,2025-07-10,0,cotizacion,"[17014, '[DAR02523025] DA2523 FILTRO AIRE CONICO FREIGHTLINER (025 DA2523)']",[DAR02523025] DA2523 FILTRO AIRE CONICO FREIGHTLINER (025 DA2523),[CC222222222221] CONSUMIDOR FINAL CALLE 6,2,SUCURSAL CALLE 6,7,2025,10,1
276107,DAB02975025,1,2025-07-10,0,cotizacion,"[17294, '[DAB02975025] DA2975 FILTRO AIRE CHEV. LUV DMAX DIESEL 2.5 Y 3.0 (025 DA2975)']",[DAB02975025] DA2975 FILTRO AIRE CHEV. LUV DMAX DIESEL 2.5 Y 3.0 (025 DA2975),[CC222222222221] CONSUMIDOR FINAL CALLE 6,2,SUCURSAL CALLE 6,7,2025,10,1
276117,DAE04103025,1,2025-07-10,0,cotizacion,"[17462, '[DAE04103025] DA4103 FILTRO AIRE 2o KOMATSU (025 DA4103)']",[DAE04103025] DA4103 FILTRO AIRE 2o KOMATSU (025 DA4103),[CC222222222221] CONSUMIDOR FINAL CALLE 6,2,SUCURSAL CALLE 6,7,2025,10,1
276118,DAE04109025,1,2025-07-10,0,cotizacion,"[17466, '[DAE04109025] DA4109 FILTRO AIRE 2o. KOMATSU OTRAS. (025 DA4109)']",[DAE04109025] DA4109 FILTRO AIRE 2o. KOMATSU OTRAS. (025 DA4109),[CC222222222221] CONSUMIDOR FINAL CALLE 6,2,SUCURSAL CALLE 6,7,2025,10,1


# DESCGLOSAR NOMBRE

In [18]:
import ast

def get_second(val):
    try:
        # convierte el string en lista real de Python
        lista = ast.literal_eval(str(val))
        if isinstance(lista, list) and len(lista) > 1:
            return lista[1]  # devuelve el segundo elemento
    except Exception:
        return None
    return None

vp_def["product_id"] = vp_def["product_id"].apply(get_second)

import re

def split_cliente(val):
    if val is None:
        return None, None
    text = str(val)
    match = re.match(r"\[(.*?)\]\s*(.*)", text)
    if match:
        return match.group(1), match.group(2)  # (ID, Nombre)
    return None, text

# Crear dos columnas nuevas
vp_def = vp_def.copy()

vp_def.loc[:, "cliente_id"], vp_def.loc[:, "cliente_name"] = zip(*vp_def["cliente_name"].apply(split_cliente))
vp_def = vp_def.drop(columns=["cliente_id"])

/tmp/ipykernel_118733/3119152225.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vp_def["product_id"] = vp_def["product_id"].apply(get_second)


In [19]:
import re

def split_producto(val):
    if val is None:
        return None, None, None
    text = str(val).strip()
    
    # buscar con regex: [producto] descripcion (codigo)
    match = re.match(r"\[(.*?)\]\s*(.*)\((.*?)\)\s*$", text)
    if match:
        producto = match.group(1).strip()
        descripcion = match.group(2).strip().rstrip(",")   # quitamos coma final si la hay
        codigo_interno = match.group(3).strip()
        return producto, descripcion, codigo_interno
    
    return None, text, None  # fallback: si no cumple el patrón

# aplicar y crear nuevas columnas
vp_def.loc[:, ["producto", "descripcion", "codigo_interno"]] = list(vp_def["product_id"].apply(split_producto))
vp_def = vp_def.drop(columns=["product_id"], errors="ignore")


In [20]:
vp_def.sample(10)

,store_name,origen,cliente_name,fecha,vp,mes,dia,ano,producto,descripcion,codigo_interno
78498,PRINCIPAL COTA,cotizacion,CUMMINS DE LOS ANDES S.A.,2025-02-07,8,2,7,2025,DAB08191025,DA8191 FILTRO AIRE FOTON AUMARK BJ1126,025 DA8191
335321,SUCURSAL MEDELLIN,cotizacion,CATERCOL S.A.,2025-08-25,1,8,25,2025,DAB08239025,DA8239 FILTRO DE AIRE - VOLKSWAGEN,025 DA8239
94030,PRINCIPAL COTA,cotizacion,PROMOCION BALDWIN >$1´000.000 (NO FACTURAR A ESTE CLIENTE),2025-02-18,1,2,18,2025,BCS20168125,GS168B FILTRO COMBUSTIBLE SEPARADOR CUMMINS M-11,125 BF1282
160914,SUCURSAL CALLE 6,cotizacion,CONSUMIDOR FINAL CALLE 6,2025-04-07,1,4,7,2025,DAP04020025,DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR,025 DA4020
148171,SUCURSAL BARRANQUILLA,pos,DIGABE DEL CARIBE SAS,2025-03-28,2,3,28,2025,DAE08142025,"DA8142 FILTRO AIRE CATERPILLAR, CLASS, FEND, WIRTGEN",025 DA8142
8702,SUCURSAL VALLADOLID,pos,R & M DE COLOMBIA S.A.S.,2024-12-06,21,12,6,2024,BLE00033125,G033 FILTRO ACEITE DETROIT,125 P20HD
113884,PRINCIPAL COTA,cotizacion,MARIO ALBERTO HUERTAS COTES,2025-03-04,1,3,4,2025,BLE00999125,G999 FILTRO ACEITE - DAF - KENWORTH,125 P40019
261124,PRINCIPAL COTA,cotizacion,CONSUMIDOR FINAL COTA,2025-06-27,12,6,27,2025,DHE00317137,G317 FILTRO DIRECCION HIDRAULICA CAMIONES,137 RP352
77256,SUCURSAL BARRANQUILLA,cotizacion,CUMMINS DE LOS ANDES S.A.,2025-02-06,4,2,6,2025,BWS20081125,GS081A23 FILTRO SELLADO PARA AGUA RADIADOR,125 BW5076
165587,SUCURSAL MEDELLIN,cotizacion,CONSUMIDOR FINAL ITAGUI,2025-04-10,1,4,10,2025,DCE01020189,G1020 FILTRO COMBUSTIBLE - FOTON,189 G1020


In [21]:
import pandas as pd

# Asegurar tipo fecha
#vp_def["fecha"] = pd.to_datetime(vp_def[["ano","mes","dia"]])

# Crear columna de Mes (primer día del mes)
vp_def["Mes"] = vp_def["fecha"].dt.to_period("M").dt.to_timestamp()

# Agrupar por sucursal + producto + mes
vp_mensual = (
    vp_def.groupby(["store_name","producto","Mes"], as_index=False)
          .agg({"vp": "sum"})
          .rename(columns={"store_name": "Sucursal",
                           "producto": "Producto",
                           "vp": "Ventas_perdidas"})
)
